### CS 378: Automated Planning for Robots
# HW 1: Modeling Classical Planning Problems in Unified Planning Framework
### Spring 2026
### Lecturer: Erez Karpas
### TA: Talal Ayman

### Students: Akshita Santra (as234898), Muyang Zhou (mz9939)

In this homework assignment you will practice modeling a problem as a classical planning problem.

You will use the [Unified Planning Framework](https://github.com/aiplan4eu/unified-planning), so the first step is to make sure the unified planning framework is installed.
We are also going to install [Pyperplan](https://github.com/aibasel/pyperplan), a lightweight Python planner, which will allow us to solve planning problems.

We must also make sure to import the package.

In [ ]:
%pip install unified-planning[pyperplan]

import unified_planning
from unified_planning.shortcuts import *

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\karpa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


We will now create a simple planning problem with, to make sure everything is working correctly.

The problem will have one variable (x), one action (a) which turns x to true, an initial state of x=false, and a goal of x=true.


In [ ]:
simple_problem = Problem('simple_problem')
x = Fluent('x', BoolType())

a = InstantaneousAction('a')
a.add_effect(x, True)

simple_problem.add_fluent(x)
simple_problem.add_action(a)
simple_problem.set_initial_value(x, False)
simple_problem.add_goal(x)

print(simple_problem)

problem name = simple_problem

fluents = [
  bool x
]

actions = [
  action a {
    preconditions = [
    ]
    effects = [
      x := true
    ]
  }
]

initial fluents default = [
]

initial values = [
  x := false
]

goals = [
  x
]




We can now call solve our problem, by using the OneShotPlanner mode.

In [ ]:
planner = OneshotPlanner(name='pyperplan')
result = planner.solve(simple_problem)
print(result)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 563 of `C:\Users\karpa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: pyperplan
  * Developers:  Albert-Ludwigs-Universität Freiburg (Yusra Alkhazraji, Matthias Frorath, Markus Grützner, Malte Helmert, Thomas Liebetraut, Robert Mattmüller, Manuela Ortlieb, Jendrik Seipp, Tobias Springenberg, Philip Stahl, Jan Wülfing)
  * Description: Pyperplan is a lightweight STRIPS planner written in Python.

status: SOLVED_SATISFICING
engine: Pyperplan
plan: SequentialPlan:
    a


Now that we have made sure everything is working, we can start describing the homework assignment

##  Homework Assignment: Modeling Control of Home Service Robots as a Classical Planning Problem

### Part1: Modeling
In this homework assignment, you will model a planning problem which can plan for a set of home service robot.

We have different kinds of robots:
* Mobile manipulator - a mobile robot which can move between rooms, and tidy up a room. However, the mobile manipulator has dirt on its wheelrs, so after a mobile manipulator tidies up a room, it is no longer clean.
* Mobile vacuum - a mobile robot which can move between rooms, and vacuum the room it is in. A room can only be vacuumed after it has been tidied up. After a room has been vacuumed it is clean.

Finally, rooms are connected according to some connectivity map.
The goal is to have all rooms clean and tidy.

Your assignment is to model the given problem using the Unified Planning Framework.
You will need to solve the problem for the following maps:

Map 1:
* 2 connected rooms (rm1 and rm2)
* 1 mobile manipulator, starts in rm1
* 1 vacuum, starts in rm1
* both rooms start not clean and not tidy

Map2:
* 4 rooms - nw, ne, sw, se (4 corners). Each room is connected to two adjacent rooms, and not to the one in the opposite corner (for example, nw is connected to ne and sw, but not to se).
* 1 mobile manipulator, starts in sw
* 1 vacuum, starts in ne
* all rooms start not clean and not tidy

Map3:
* 9 rooms: corridor (which is a room), which is connected to 8 different rooms (rm1 -- rm8) 
* 1 mobile manipulator, starts in corridor
* 1 vacuum, starts in corridor
* rooms rm1 -- rm4 start tidy but not clean, rooms rm5 -- rm8 start clean but not tidy.

In the following code block, you will need to define 3 functions: ``prob1(), prob2(), prob3()`` 
Each of these functions should return a problem object which corresponds to the map described above.

The code block below will call your functions to generate the problem, and then call a planner to solve them and print the solutions.


In [ ]:
# Solution
room = UserType('room')
mobile_robot = UserType('mobile_robot')
vacuum_robot = UserType('vacuum_robot', mobile_robot)
mobile_manipulator = UserType('mobile_manipulator', mobile_robot)

robot_at = Fluent('robot-at', r=mobile_robot, rm=room)
tidy = Fluent('tidy-room', rm=room)
clean = Fluent('clean-room', rm=room)
connected = Fluent('connected', rm1=room, rm2=room)

move = InstantaneousAction('move', r=mobile_robot, rm_from=room, rm_to=room)
move_r = move.parameter('r')
move_from = move.parameter('rm_from')
move_to = move.parameter('rm_to')
move.add_precondition(robot_at(move_r, move_from))
move.add_precondition(connected(move_from, move_to))
move.add_effect(robot_at(move_r, move_from), False)
move.add_effect(robot_at(move_r, move_to), True)

tidy_action = InstantaneousAction('tidy', r=mobile_manipulator, rm=room)
tidy_r = tidy_action.parameter('r')
tidy_rm = tidy_action.parameter('rm')
tidy_action.add_precondition(robot_at(tidy_r, tidy_rm))
tidy_action.add_effect(tidy(tidy_rm), True)
tidy_action.add_effect(clean(tidy_rm), False)

clean_action = InstantaneousAction('clean', r=vacuum_robot, rm=room)
clean_r = clean_action.parameter('r')
clean_rm = clean_action.parameter('rm')
clean_action.add_precondition(robot_at(clean_r, clean_rm))
clean_action.add_precondition(tidy(clean_rm))
clean_action.add_effect(clean(clean_rm), True)

problem = Problem('robot_cleaning')
problem.add_fluent(robot_at)
problem.add_fluent(tidy)
problem.add_fluent(clean)
problem.add_action(move)
problem.add_action(tidy_action)
problem.add_action(clean_action)

# Solution: map1 
map1 = problem.clone()
r1 = map1.add_object('r1', room)
r2 = map1.add_object('r2', room)
vr1 = map1.add_object('vr1', vacuum_robot)
mm1 = map1.add_object('mm1', mobile_manipulator)
map1.set_initial_value(connected(r1, r2), True)
map1.set_initial_value(connected(r2, r1), True)
map1.set_initial_value(robot_at(vr1, r1), True)
map1.set_initial_value(robot_at(mm1, r2), True)
map1.set_initial_value(tidy(r1), False)
map1.set_initial_value(tidy(r2), False)
map1.set_initial_value(clean(r1), False)
map1.set_initial_value(clean(r2), False)
map1.add_goal(clean(r1))
map1.add_goal(clean(r2))
map1.add_goal(tidy(r1))
map1.add_goal(tidy(r2))
#print(map1)

def prob1():
    return map1

# Solution: map2
map2 = problem.clone()
nw = map2.add_object('nw', room)
ne = map2.add_object('ne', room)
sw = map2.add_object('sw', room)
se = map2.add_object('se', room)
vr1 = map2.add_object('vr1', vacuum_robot)
mm1 = map2.add_object('mm1', mobile_manipulator)
map2.set_initial_value(connected(nw, ne), True)
map2.set_initial_value(connected(nw, sw), True)
map2.set_initial_value(connected(ne, nw), True)
map2.set_initial_value(connected(ne, se), True)
map2.set_initial_value(connected(se, ne), True)
map2.set_initial_value(connected(se, sw), True)
map2.set_initial_value(connected(sw, se), True)
map2.set_initial_value(connected(sw, nw), True)

map2.set_initial_value(robot_at(vr1, ne), True)
map2.set_initial_value(robot_at(mm1, sw), True)
map2.set_initial_value(tidy(nw), False)
map2.set_initial_value(tidy(ne), False)
map2.set_initial_value(tidy(sw), False)
map2.set_initial_value(tidy(se), False)
map2.set_initial_value(clean(nw), False)
map2.set_initial_value(clean(ne), False)
map2.set_initial_value(clean(sw), False)
map2.set_initial_value(clean(se), False)
map2.add_goal(clean(nw))
map2.add_goal(clean(ne))
map2.add_goal(clean(sw))
map2.add_goal(clean(se))
map2.add_goal(tidy(nw))
map2.add_goal(tidy(ne))
map2.add_goal(tidy(sw))
map2.add_goal(tidy(se))
#print(map2)

def prob2():
    return map2

# Solution: map3
map3 = problem.clone()
corridor = map3.add_object('corridor', room)
rms = [map3.add_object(f'r{i}', room) for i in range(1, 9)]
vr1 = map3.add_object('vr1', vacuum_robot)
mm1 = map3.add_object('mm1', mobile_manipulator)

for i in range(1, 9):
    map3.set_initial_value(connected(corridor, rms[i-1]), True)
    map3.set_initial_value(connected(rms[i-1], corridor), True)

    map3.set_initial_value(clean(rms[i-1]), i >= 5)
    map3.set_initial_value(tidy(rms[i-1]), i < 5)
                           
    map3.add_goal(clean(rms[i-1]))
    map3.add_goal(tidy(rms[i-1]))

map3.set_initial_value(robot_at(vr1, corridor), True)
map3.set_initial_value(robot_at(mm1, corridor), True)

def prob3():
    return map3

The code below will call your functions to generate the problems, and then try to solve them using Pyperplan.

In [ ]:
from unified_planning.engines import PlanGenerationResultStatus

def solve(prob):
    planner = OneshotPlanner(name='pyperplan')
    result = planner.solve(prob)
    if result.status in [PlanGenerationResultStatus.SOLVED_SATISFICING,
                         PlanGenerationResultStatus.SOLVED_OPTIMALLY]:
        print("SOLVED. Plan length:", len(result.plan.actions), "Plan:", result.plan.actions)
    else:
        print("NOT SOLVED")
    

print("Map1")
solve(prob1())

print("Map2")
solve(prob2())

print("Map3")
solve(prob3())


Map1
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 563 of `C:\Users\karpa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: pyperplan
  * Developers:  Albert-Ludwigs-Universität Freiburg (Yusra Alkhazraji, Matthias Frorath, Markus Grützner, Malte Helmert, Thomas Liebetraut, Robert Mattmüller, Manuela Ortlieb, Jendrik Seipp, Tobias Springenberg, Philip Stahl, Jan Wülfing)
  * Description: Pyperplan is a lightweight STRIPS planner written in Python.

SOLVED. Plan length: 6 Plan: [tidy(mm1, r2), move(mm1, r2, r1), tidy(mm1, r1), clean(vr1, r1), move(vr1, r1, r2), clean(vr1, r2)]
Map2
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 563 of `C:\Users\karpa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\unified_plann

### Part 2: Questions

Please answer each question in the markdown cell below it.
If your answer can be justified using some python code, feel free to also add a code block.

#### Q1

Are the solutions returned by Pyperplan for all problems optimal?
If so, explain.
If not, give a shorter plan to one of the problems.


#### Answer 1


The plan below for map2 is shorter than the one returned by Pyperplan.



In [ ]:
from unified_planning.plans import SequentialPlan
from unified_planning.engines.sequential_simulator import *

map2 = prob2()
se = map2.object('se')
sw = map2.object('sw')
ne = map2.object('ne')
nw = map2.object('nw')
vr1 = map2.object('vr1')
mm1 = map2.object('mm1')

plan = SequentialPlan([tidy_action(mm1, sw), move(mm1, sw, se), 
        tidy_action(mm1, se), move(mm1, se, ne), 
        tidy_action(mm1, ne), move(mm1, ne, nw), 
        tidy_action(mm1, nw),
        clean_action(vr1, ne), move(vr1, ne, se), 
        clean_action(vr1, se), move(vr1, se, sw),
        clean_action(vr1, sw), move(vr1, sw, nw),
        clean_action(vr1, nw)])
print(len(plan.actions), "actions in the plan:")

14 actions in the plan:


#### Q2

Assuming the rooms are all strongly connected, and there is at least one vacuum and at least one mobile manipulator, is it possible to create an unsolvable instance of the above problem?
If so, give one.
If not, explain why.


#### Answer 2



It is always possible to come up with a plan where the mobile manipulator goes through all the rooms first, and then the vacuum goes through all the rooms.
Therefore, there will always be a solution.

#### Q3

Assuming there are $n$ rooms, which all start being not clean and not tidy, and assuming there are $m$ robots of each type, what would be the cost of an optimal plan in the best case (that is, you can pick the connectivity of the rooms and the initial positions of the robots)?

Answer for the case of $m=1$, $m=2$, $m=n/2$, and $m=n$.

#### Answer 3

In all cases, there must be $n$ tidy actions and $n$ clean actions.

When $m=1$, there must also be $n-1$ move actions for each robot, for a total of $4m-2$

When $m=2$, each robot can cover half the rooms, so there must be $n/2-1$ move actions for each of the 4 robots, for a total of $4m-2$

When $m=n/2$, each robot need only cover 2 rooms, so it need only move once for a total of $n$ moves, and $3n$ actions in total.

When $m=n$, each robot does not need to move, so we only need $2n$ actions.